In [1]:
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv

In [35]:
def pull_from_csv(csv_file, row=3, header=1):
        lwp_ind = str(csv_file).lower().find('lwp')
        lwp = str(csv_file)[lwp_ind:lwp_ind+14]
        df = pd.read_csv(csv_file, header=[header], on_bad_lines='skip')
        try:
            return [lwp,*df.iloc[row].name]
        except:
            return [lwp,*df.iloc[row]]
        
        
        
        
        
def met_ratio(df, up, down, error_method='Cramer', error_label=' Error (SD)', output='both'):
    if not isinstance(up,list):
        up = [up]
    if not isinstance(down,list):
        down = [down]    
   
    up_err_name = [f'{item}{error_label}' for item in up]
    down_err_name = [f'{item}{error_label}' for item in down]
    for item in np.concatenate([up_err_name, down_err_name]).flat:
        if item not in df.columns:
            raise KeyError(f'Cannot find "{item}" in the DataFrame. Please check the given name is correct.')
    
    up_amp = df[up].astype(float).sum(axis=1)
    down_amp = df[down].astype(float).sum(axis=1)
    if error_method.lower() == 'cramer':
        up_err = (df[up_err_name].astype(float)**2).sum(axis=1) ** 0.5
        down_err = (df[down_err_name].astype(float)**2).sum(axis=1) ** 0.5
    elif error_method.lower()=='narrow':
        up_err = df[up_err_name].astype(float).max(axis=1) 
        down_err = df[down_err_name].astype(float).max(axis=1) 
    else:
        raise KeyError('Please choose "Cramer" or "Narrow" for the error_method')
    
    ratio = up_amp / down_amp
    ratio_err = ratio * ((up_err/up_amp)**2 + (down_err/down_amp)**2)**0.5 
    
    if len(up)>1:
        up_label =f'({"+".join(up)})'
    else:
        up_label = f'{"+".join(up)}'
    if len(down)>1:
        down_label = f'({"+".join(down)})'
    else:
        down_label = f'{"+".join(down)}'    
    col_name = f'{up_label}/{down_label}'   
    col_error_name = f'{col_name} Error (SD) {error_method}'
    
    if output=='both':
        return pd.DataFrame({col_name: ratio, col_error_name: ratio_err})
    elif output=='ratio':
        return pd.DataFrame({col_name: ratio})      
    elif output=='error':
        return pd.DataFrame({col_error_name: ratio_err})   
    else:
        raise KeyError('Please choose "ratio", "error" or "both" (default) as output')

In [36]:
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY')
# data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/Ellie')
# data_folder = Path('/Users/papo/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA')
data_folder = Path('/Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA')

# basis_folder = Path('/Users/papo/Sync/Projects/PAINT_MRS_CSI/3_0T_basis_threonine_no_MM')
basis_folder = Path('/Users/patxi/Sync/Projects/PAINT_MRS_CSI/3_0T_basis_threonine_no_MM')

sdat_files = [f for f in sorted(data_folder.rglob('*')) if ("act.sdat" in f.name.lower() and "csi" in f.name.lower())]
for file in tqdm(sdat_files):
    ref = Path(f'{str(file)[0:-8]}ref.SDAT')
    csv = Path(f'{str(file)[0:-5]}_both.csv')
    pdf = Path(f'{str(file)[0:-4]}pdf')
    # print(csv)
    if not csv.is_file():
        # command = f'tarquin --rows 3 --cols 2 --input {file} --output_pdf {pdf} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        command = f'tarquin --rows 3 --cols 2 --input {file} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        print(f'{command}\n') 
        os.system(command)


100%|██████████| 48/48 [00:00<00:00, 23032.44it/s]


In [43]:
csv_files = [f for f in sorted(data_folder.rglob('*'), key=lambda x:x.parent.parent.stem[4:6]) if ("act_both.csv" in f.name.lower() and "csi" in f.name.lower())]



rows_to_process = {'BGT': 5, #(BGT = Basal Ganglia and Thalamic, ie deep GM)
                   'WM': 3,
                   }

for label,row_num in rows_to_process.items():
    data_list = []
    for file in csv_files:
        data_list.append([*pull_from_csv(file,row=row_num),*pull_from_csv(file,row=row_num+8),*pull_from_csv(file,header=16,row=row_num+1)])
    my_columns = list(pd.read_csv(file, header=[1], on_bad_lines='skip'))
    my_errors = [item+' Error (SD)' for item in my_columns]
    my_diagnostics = pull_from_csv(file,row=0, header=16)    
    df = pd.DataFrame(data_list, columns=['LWP', *my_columns, 'Error for LWP', *my_errors,*my_diagnostics ])
    df.to_excel(f'{label}.xlsx', index=False)
    
    #now calculate ratios
    df_L2N = met_ratio(df,['Lac', 'Threonine'], 'TNAA')
    df_L2N_narrow_error = met_ratio(df,['Lac', 'Threonine'], 'TNAA', error_method='Narrow', output='error')
    df_N2C = met_ratio(df,'TNAA', 'TCho')
    df_C2C = met_ratio(df,'TCho', 'Cr')
    df_N2Cr = met_ratio(df,'TNAA', 'Cr')


    # df_ratio = pd.concat((df_L2N, df_N2C, df_C2C, df_N2C, df), axis=1)
    df_ratio = pd.concat((df_L2N,df_L2N_narrow_error, df_N2C, df_C2C, df_N2Cr, df), axis=1)

    df_ratio.set_index('LWP').to_excel(f'{label}_RATIOS_INSPIRE.xlsx')



In [44]:
dfbb = met_ratio(df,'TNAA', 'TCho', error_method='Narrow', output='error')
dfbb


,TNAA/TCho Error (SD) Narrow
0,1.156009
1,3.540930
2,1.541871
3,1.179148
4,1.038445
5,0.701964
6,1.713380
7,2.189811
8,0.541786
9,1.202693


In [45]:
data_list

[['LWP767_evening',
  '3',
  '5',
  '1',
  '2.527060e+00',
  '8.809023e+00',
  '0.000000e+00',
  '1.266816e+01',
  '5.677337e+00',
  '0.000000e+00',
  '7.512443e+00',
  '0.000000e+00',
  '1.349072e+01',
  '5.419380e+00',
  '6.129886e+00',
  '3.285128e+00',
  '2.174229e+01',
  '1.828718e+01',
  '7.565245e+00',
  '0.000000e+00',
  '2.777832e+00',
  '2.085620e+01',
  '7.439636e+00',
  '4.002947e+01',
  '7.565245e+00',
  '1.349072e+01',
  'LWP767_evening',
  '3',
  '5',
  '1',
  '4.144270e+00',
  '3.407602e+00',
  '4.785465e+00',
  '3.186666e+00',
  '5.489719e+00',
  '3.096934e+00',
  '6.591963e+00',
  '4.426750e+00',
  '7.307418e+00',
  '3.895925e+00',
  '1.235927e+01',
  '4.409958e+00',
  '7.773428e+00',
  '7.670587e+00',
  '3.458840e+00',
  '1.322761e+00',
  '1.396073e+00',
  '7.097460e+00',
  '6.140814e+00',
  '5.312770e+00',
  '1.312889e+00',
  '6.857713e+00',
  'LWP767_evening',
  '3',
  '5',
  '1',
  '1.227610e+00',
  '5.236069e+01',
  '9.190007e+01',
  '0.000000e+00',
  '7.450182e-

In [46]:
my_columns

['Row',
 'Col',
 'Slice',
 '-CrCH2',
 'Ala',
 'Asp',
 'Cr',
 'GABA',
 'GPC',
 'Glc',
 'Gln',
 'Glu',
 'Gua',
 'Ins',
 'Lac',
 'NAA',
 'NAAG',
 'PCh',
 'Prop_Glycol',
 'Scyllo',
 'Tau',
 'Threonine',
 'TNAA',
 'TCho',
 'Glx']

In [47]:
df

,LWP,Row,Col,Slice,-CrCH2,Ala,Asp,Cr,GABA,GPC,...,water freq (Hz),WNR,res water amp,res water freq,baseline dev,max baseline,min baseline,baseline shape,initial residual,final residual
0,LWP767_evening,3,5,1,2.527060e+00,8.809023e+00,0.000000e+00,1.266816e+01,5.677337e+00,0.000000e+00,...,2.807617e+00,1.494027e+03,4.993216e-02,1.953125e+00,1.927997e-02,5.829514e-02,-4.005583e-02,-2.660896e-02,7.018114e+00,6.191645e+00
1,LWP762_morning,3,5,1,0.000000e+00,1.425245e+01,2.152494e+01,4.126797e+00,0.000000e+00,1.924082e+00,...,-2.197266e+00,1.058467e+03,6.450923e-02,9.765625e-01,7.649172e-02,1.532105e-01,-1.594774e-01,-9.726559e-02,8.030071e+00,7.642780e+00
2,LWP772_morning,3,5,1,1.805597e+00,2.135336e+00,7.592475e+00,1.305497e+01,1.059296e+01,1.955434e+00,...,-7.324219e-01,1.829506e+03,5.263130e-02,-0.000000e+00,2.381434e-02,5.052477e-02,-5.469399e-02,1.209620e-02,6.511388e+00,5.780941e+00
3,LWP774_evening,3,5,1,2.156436e+01,1.047241e+01,2.109410e+01,1.964967e+01,9.060235e+00,1.727141e+00,...,4.882812e-01,1.274908e+03,3.493262e-02,1.098633e+00,2.194881e-02,3.826059e-02,-4.053080e-02,-1.814630e-02,5.969932e+00,5.403073e+00
4,LWP719_morning,3,5,1,1.742017e+00,8.697905e+00,0.000000e+00,1.068699e+01,0.000000e+00,4.597246e+00,...,-3.662109e-01,1.686164e+03,2.829115e-01,2.441406e-01,5.138145e-02,1.107908e-01,-7.631279e-02,2.010491e-02,7.043264e+00,6.408933e+00
5,LWP766_evening,3,5,1,7.918909e+00,6.398566e+00,2.355884e+01,1.308431e+01,7.221846e+00,4.655246e+00,...,-4.882812e-01,1.476873e+03,1.063831e-02,9.765625e-01,2.390900e-02,5.677018e-02,-4.954149e-02,-6.876975e-03,6.843388e+00,6.288174e+00
6,LWP776_evening,3,5,1,2.947400e+00,0.000000e+00,4.335862e+00,3.042977e+00,0.000000e+00,3.472197e+00,...,-1.098633e+00,1.401277e+03,1.034921e-01,8.544922e-01,4.110517e-02,7.849967e-02,-8.747300e-02,-9.865180e-03,7.735658e+00,7.468980e+00
7,LWP718_morning,3,5,1,5.274860e+00,0.000000e+00,6.348680e+00,8.251790e+00,3.366255e+00,3.565437e+00,...,-7.324219e-01,1.868764e+03,2.857342e-02,1.220703e-01,4.054027e-02,7.809283e-02,-9.555740e-02,-7.464514e-03,7.309107e+00,6.560452e+00
8,LWP764_evening,3,5,1,6.713894e+00,5.663672e+00,3.241326e+00,1.500741e+01,0.000000e+00,1.024077e+01,...,1.220703e-01,1.214777e+03,3.269567e-02,2.197266e+00,2.549250e-02,5.272000e-02,-5.338116e-02,-1.550889e-02,6.598788e+00,5.246114e+00
9,LWP711_morning,3,5,1,5.177247e+00,0.000000e+00,7.296516e+00,1.398440e+01,0.000000e+00,3.854052e+00,...,-3.417969e+00,1.228555e+03,1.892215e-01,-8.544922e-01,2.419431e-02,4.345174e-02,-7.057038e-02,-1.944067e-02,6.906374e+00,6.115927e+00


In [53]:
file
# row_num
# pull_from_csv(file, row=row_num )

PosixPath('/Users/patxi/Sync/Projects/PAINT_MRS/INSPIRE_MRS_DATA/LWP767_morning/C6530178_WIP_CSI_PB_auto_TR2S_SENSE_12_2_raw_act_both.csv')